In [ ]:
import util
import numpy as np
from numpy.typing import NDArray
import matplotlib.pyplot as plt

In [ ]:
smp_a = np.random.uniform(-1,1,100000)
smp_b = np.imag(np.exp((100j+1)*np.linspace(0,4,1024)))
smp_b /= smp_b.sum()

conv_groundtruth = np.convolve(smp_a,smp_b,mode="full")

plt.plot(conv_groundtruth)
util.display_sound(conv_groundtruth)

In [ ]:
conv_groundtruth.size,(smp_a.size+smp_b.size-1)

In [ ]:
def conv_circular(a: NDArray, b: NDArray) -> NDArray:
    assert len(a.shape) == 1 and len(b.shape) == 1
    if b.size > a.size:
        return conv_circular(b, a)
    if b.size < a.size:
        return conv_circular(a, np.concatenate([b, np.zeros(a.size-b.size)]))

    return np.fft.ifft(np.fft.fft(a) * np.fft.fft(b))
    # np.real(conv_circular(
    #     np.array([1,2,3,4,5,6,7,8,9]),
    #     np.array([1,10,100,1000]),
    # ))[4-1:]
    # # -> array([1234., 2345., 3456., 4567., 5678., 6789.])

def conv_overlap_scrap(signal: NDArray, filter: NDArray) -> NDArray:
    N = 2048
    assert len(signal.shape) == 1 
    assert len(filter.shape) == 1 
    assert filter.size <= N//2
    chunk_size = N - filter.size + 1
    signal_len_in = signal.size
    signal = np.concatenate([np.zeros(filter.size - 1), signal, np.zeros(filter.size - 1)])
    signal = np.concatenate([signal, np.zeros((signal.size+chunk_size-1)//chunk_size*chunk_size-signal.size)])
    signal_out = []
    for i in range(signal.size//chunk_size):
        signal_out.append(
            np.real(conv_circular(
                signal[i*chunk_size:i*chunk_size+N],
                filter,
            )[filter.size-1:])
        )
    return np.concatenate(signal_out)[:signal_len_in + filter.size - 1]

conv_fast = conv_overlap_scrap(smp_a,smp_b)

print("rmse error from ground truth: ", np.sqrt(np.mean((conv_fast - conv_groundtruth)**2)))
plt.plot(conv_fast)
plt.plot(conv_fast-conv_groundtruth)
util.display_sound(conv_fast)

In [ ]:
smp_a.size + smp_b.size - 1

In [ ]:
np.concatenate([
    [1,2,3],
    np.zeros(5-3)
])

In [ ]:
l = 32
# latency = size - filter_size
i = 0
while i < 10:
    s = l*2
    # filter_size = size - latency
    f = s - l
    for _ in range(1):
        i += 1
        l += f
        print(l,s,f, f/s)